# Phenology and branch movement

This notebook demonstrated the preparation of the 4DGeo phenology example. The dataset contains weekly scans of a tree over a time period of 6 weeks, with the first scan on 27 March 2025 and the last scan on 9 May 2025.

The starting point of the notebook are pointwise motion vectors computed for all pairs of consecutive scans. These may be computed using tools like [PlantMove](https://doi.org/10.1016/j.jag.2022.102781). Here, we read these motion vectors from a .npz file with Python. In addition, we have the .laz files for the individual scans, which for each epoch are split into the branch of interest and the rest of the tree. 

## Imports

In [1]:
from pathlib import Path
import vapc
import numpy as np
from scipy.spatial import KDTree
import pandas as pd
try:
    import pyvista as pv
    pv.set_jupyter_backend('trame')  # or switch to 'trame' instead of 'html'
    visualization = True
except:
    visualization = False
import laspy

import os
import sys
sys.path.insert(0, "../src")
from fourdgeo import projection
from fourdgeo import utilities
from fourdgeo import change
visualization = False

## Getting the data

In [2]:
import os
import zipfile, urllib.request, shutil

data_url = "https://heibox.uni-heidelberg.de/f/de3dc527cb3f478f896e/?dl=1"
file_name = "branch_evolution.zip"

with urllib.request.urlopen(data_url) as response, open(file_name, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall("data")

# Deleting the .zip file
os.remove(file_name)

## Overview of the data

Let's first get an overview of the input data. We visualize the point clouds of the first and last scan side by side with `pyvista`, coloured by height.

Use the `ctrl` and `shift` keys to interact with the point cloud.

In [3]:
laz_files_branches = list(Path("data").glob("*_branch.laz"))
laz_files_rest = list(Path("data").glob("*_rest.laz"))

las_b0 = laspy.read(laz_files_branches[0])
pc_b0 = np.c_[las_b0.x, las_b0.y, las_b0.z]
las_b6 = laspy.read(laz_files_branches[6])
pc_b6 = np.c_[las_b6.x, las_b6.y, las_b6.z]
las_r0 = laspy.read(laz_files_rest[0])
pc_r0 = np.c_[las_r0.x, las_r0.y, las_r0.z]
las_r6 = laspy.read(laz_files_rest[6])
pc_r6 = np.c_[las_r6.x, las_r6.y, las_r6.z]

if visualization:
    pl = pv.Plotter(notebook=True, shape=(1, 2))
    pl.add_points(pc_b0[::5], scalars = pc_b0[::5, 2], point_size=3, render_points_as_spheres=True)
    pl.add_points(pc_r0[::5], scalars = pc_r0[::5, 2], point_size=3, render_points_as_spheres=True)
    pl.add_text("27 March 2025", position='upper_left', font_size=10, color='black')
    pl.subplot(0, 1)
    pl.add_points(pc_b6[::5], scalars = pc_b6[::5, 2], point_size=3, render_points_as_spheres=True)
    pl.add_points(pc_r6[::5], scalars = pc_r6[::5, 2], point_size=3, render_points_as_spheres=True)
    pl.add_text("09 May 2025", position='upper_left', font_size=10, color='black')

    pl.link_views()
    pl.remove_scalar_bar()
    pl.camera_position = 'xz'
    pl.camera.zoom(2)
    pl.show()

The motion vectors have been computed with [PlantMove](https://doi.org/10.1016/j.jag.2022.102781) for each point in a selected branch in the point cloud. They are stored in a .npz archive. Let's look at the structure for the first observation (change between first epoch on 27 March and second epoch on 4 April 2025).

`cloud_x` is the first time step, \
`cloud_y` is the second time step, \
`motionfield` is the derived motion field \
and `cloud_new` is the point cloud of the second time step registered onto the first time step by applying the motion vectors.

In [4]:
files = list(Path("data").glob("*.npz"))
obs_file = files[0]
data = np.load(obs_file)
cloud_x = data["x_original"]  # t0
cloud_y = data["y_original"]  # t1
cloud_new = data["y_new"]  # t1 to t0
motionfield = data["motionfield"]  # motion field from t0 to t1
motion_vec = motionfield[:, 3:6]
motion_magnitudes = np.linalg.norm(motion_vec, axis=1)

In [5]:
print("Motion magnitudes between t0 and t1:\n")
print(f"{'Min:':20} {motion_magnitudes.min():.3f} m")
print(f"{'Max:':20} {motion_magnitudes.max():.3f} m")
print(f"{'Mean:':20} {motion_magnitudes.mean():.3f} m")
print(f"{'Standard deviation:':20} {motion_magnitudes.std():.3f} m")

Motion magnitudes between t0 and t1:

Min:                 0.000 m
Max:                 0.018 m
Mean:                0.005 m
Standard deviation:  0.003 m


## Projecting the point clouds onto 2D images

For each observation, we create a 2D image of the point cloud by projecting the 3D points onto a 2D plane from the viewpoint of the scanner.

### Prepare the configuration file

In [6]:
configuration = {
    "project_setting": {
        "project_name": "Branch_evolution",
        "output_folder": "./out",
        "temporal_format": "%y%m%d_%H%M%S",
        "silent_mode": True,
        "include_timestamp": False
    },
    "pc_projection": {
        "pc_path": "",
        "make_range_image": False,
        "make_color_image": True,
        "top_view": False,
        "save_rot_pc": False,
        "resolution_cm": 0.2,
        "camera_position": [
            475923.4099,
            5473923.7662,
            159.1088
        ],
        "rgb_light_intensity": 100,
        "range_light_intensity": 60,
        "epsg": None
    }
}

### Merging each branch with its respective tree

In [7]:
from vapc.las_split_append_merge import las_merge

laz_files_rest = sorted(laz_files_rest)
laz_files_branches = sorted(laz_files_branches)

pcs = []

for enum in np.arange(len(laz_files_rest)-1):
    pos = str(laz_files_rest[enum]).find("_icp_rest")
    new_file_path = str(laz_files_rest[enum])[:pos]
    with laspy.open(laz_files_branches[enum]) as las_branch_A:
        las_branch_A = las_branch_A.read()
        las_branch_A.red = np.full(len(las_branch_A.points), 65535)
        las_branch_A.green = np.full(len(las_branch_A.points), 22768)
        las_branch_A.blue = np.full(len(las_branch_A.points), 0)
        las_branch_A.write(laz_files_branches[enum])

    with laspy.open(laz_files_branches[enum+1]) as las_branch_B:
        las_branch_B = las_branch_B.read()
        las_branch_B.red = np.full(len(las_branch_B.points), 0)
        las_branch_B.green = np.full(len(las_branch_B.points), 0)
        las_branch_B.blue = np.full(len(las_branch_B.points), 65535)
        las_branch_B.write(laz_files_branches[enum+1])

    with laspy.open(laz_files_rest[enum]) as las_rest_A:
        las_rest_A = las_rest_A.read()
        intensities = las_rest_A.intensity

        # Normalize intensities to 0-65535
        norm = (intensities - intensities.min()) / (np.ptp(intensities) + 1e-8)
        colors = (norm * 65535).astype(np.uint16)

        # Assign grayscale color based on intensity
        las_rest_A.red = las_rest_A.green = las_rest_A.blue = colors
        las_rest_A.write(laz_files_rest[enum])

    las_merge([laz_files_rest[enum], laz_files_branches[enum], laz_files_branches[enum+1]], f"{new_file_path}_{enum}_{enum+1}.laz", point_source_id=True)
    pcs.append(f"{new_file_path}_{enum}_{enum+1}.laz")

### Generating the background images

In [12]:
images = []

for enum, pc in enumerate(pcs):
    lf = laspy.read(pc)
    configuration['pc_projection']['pc_path'] = pc
    # Second projection (use reference)
    background_projection = projection.PCloudProjection(
        configuration = configuration,
        project_name = configuration['project_setting']['project_name'],
        projected_image_folder = configuration['project_setting']['output_folder'],
    )

    if enum == 0:
        (
            ref_h_fov, ref_v_fov, ref_anchor_point_xyz, 
            ref_h_img_res, ref_v_img_res
        ) = background_projection.project_pc(buffer_m = 0.5)
    else:
        background_projection.project_pc(
            ref_theta=ref_h_fov[0],
            ref_phi=ref_v_fov[0],
            ref_anchor_point_xyz=None,
            ref_h_fov=ref_h_fov,
            ref_v_fov=ref_v_fov,
            ref_h_img_res=ref_h_img_res,
            ref_v_img_res=ref_v_img_res
        )
    outfile = f"out/Branch_evolution_ColorImage_{enum}_{enum+1}.tif"
    try:
        os.rename("out/Branch_evolution_ColorImage.tif", outfile)
    except FileExistsError:
        os.remove(outfile)
        os.rename("out/Branch_evolution_ColorImage.tif", outfile)
    images.append(outfile)

In [9]:
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt

# Carousel display
def show_image(idx):
    img = plt.imread(images[idx])
    plt.figure(figsize=(6, 12))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

interact(show_image, idx=widgets.IntSlider(min=0, max=len(images)-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='idx', max=5), Output()), _dom_classes=('widget-interact'…

<function __main__.show_image(idx)>

## Deriving a subset of motion vectors by voxelization and projecting them to 2D image coordinates

Since the visualization would be too cluttered with all motion vectors, we reduce the number of motion vectors by voxelizing the point cloud and only keeping one motion vector per voxel. We then project these motion vectors onto the 2D image plane.

In [10]:
# Activate silent mode:
vapc.enable_trace(False)
vapc.enable_timeit(False)
voxel_size = 0.05

In [11]:
pl = pv.Plotter()
pl.open_gif("../docs/img/branch_evolution.gif")

colors = ["#0a2f51", "#0f596b", "#16837a", "#1d9a6c", "#56b870", "#99d492", "#deedcf"]

obs_dict = {}
for obs_id, file in enumerate(files):
    obs_dict["observation_id"] = obs_id
    obs_dict["events"] = []
    # extract time from filenames
    t_min = "_".join(file.stem.split("_")[1:3])
    t_max = "_".join(file.stem.split("_")[7:9])

    data = np.load(file)
    cloud_x = data["x_original"]  # t0
    cloud_y = data["y_original"]  # t1
    cloud_new = data["y_new"]  # t1 to t0
    motionfield = data["motionfield"]
    motion_vec = motionfield[:, 3:6]
    motion_magnitudes = np.linalg.norm(motion_vec, axis=1)

    if obs_id == 0:
        min_point = np.min(cloud_x, axis=0)
        local_origin = np.floor(min_point / 100) * 100
    else:
        pl.remove_actor(pv_cloud1)
        pl.remove_actor(pv_cloud2)
        pl.remove_actor(pv_lines)
    
    pv_cloud1 = pl.add_points(cloud_x, color=colors[obs_id], point_size=2, render_points_as_spheres=True)

    # array to dataframe
    df_y = pd.DataFrame(cloud_y, columns=["X", "Y", "Z"])
    df_new = pd.DataFrame(cloud_new, columns=["X", "Y", "Z"])

    vapc_y = vapc.Vapc(voxel_size=voxel_size, origin=local_origin.tolist())
    vapc_y.df = df_y
    vapc_y.compute = ["center_of_voxel"]
    vapc_y.compute_requested_attributes()

    vapc_new = vapc.Vapc(voxel_size=voxel_size, origin=local_origin.tolist())
    vapc_new.df = df_new
    vapc_new.compute = ["center_of_voxel"]
    vapc_new.compute_requested_attributes()
    
    if obs_id == 0:
        seed_points = np.array((vapc_new.df["center_x"].values, vapc_new.df["center_y"].values, vapc_new.df["center_z"].values)).T
        seed_points = np.unique(seed_points, axis=0)
    else:
        seed_points = pts_y_original
    
    # get nearest neighbours to seed points in cloud_new
    tree_ynew = KDTree(cloud_new)

    dist, idx_yn = tree_ynew.query(seed_points, k=1)

    # get points
    pts_y_original = cloud_y[idx_yn, :]
    pts_y_new = cloud_new[idx_yn, :]

    # create lines from nearest points to seed in cloud y and corresponding points in cloud x
    lines = np.hstack((pts_y_original, pts_y_new))

    # write json object for each line
    for i in range(lines.shape[0]):
        event_dict = {}
        event_dict["event_type"] = "motion_vector"
        event_dict["object_id"] = f"{obs_id}{i:04d}"
        event_dict["t_min"] = t_min
        event_dict["t_max"] = t_max
        event_dict["epoch_id"] = obs_id
        event_dict["change_magnitude"] = motion_magnitudes[i]
        event_dict["motion_vector_points"] = [lines[i, 0:3].tolist(),
                                              lines[i, 3:6].tolist()]
        obs_dict["events"].append(event_dict)
        # TODO @Will: modify as you like and project lines either here or in separate cell/loop

    # reshape to 2n x 3 array
    lines2plot = lines.reshape(-1, 3)

    # plot original cloud
    pv_cloud2 = pl.add_points(cloud_y, color=pv.Color(colors[obs_id+1]), point_size=2, render_points_as_spheres=True)
    pv_lines = pl.add_lines(lines2plot, color='black', width=3)
    for _ in range(5):  # to make the gif slower
        pl.write_frame()
pl.close()

![branch_evolution.gif](img/branch_evolution.gif)